In [1]:
import os
from langchain_community.vectorstores import FAISS
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_deepseek import ChatDeepSeek


In [2]:
# 导入ColBERT重排器需要的模块
from langchain.retrievers.document_compressors.base import BaseDocumentCompressor
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_core.documents import Document
from typing import Sequence
import torch
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F

/root/miniconda/envs/myrag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## ColBERT重排器

In [3]:

class ColBERTReranker(BaseDocumentCompressor):
    """ColBERT重排器"""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        model_name = "bert-base-uncased"

        # 加载模型和分词器
        object.__setattr__(self, 'tokenizer', AutoTokenizer.from_pretrained(model_name))
        object.__setattr__(self, 'model', AutoModel.from_pretrained(model_name))
        self.model.eval()
        print(f"ColBERT模型加载完成")

    def encode_text(self, texts):
        """ColBERT文本编码"""
        inputs = self.tokenizer(
            texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128
        )

        with torch.no_grad():
            outputs = self.model(**inputs)

        embeddings = outputs.last_hidden_state
        embeddings = F.normalize(embeddings, p=2, dim=-1)

        return embeddings

    def calculate_colbert_similarity(self, query_emb, doc_embs, query_mask, doc_masks):
        """ColBERT相似度计算（MaxSim操作）"""
        scores = []

        for i, doc_emb in enumerate(doc_embs):
            doc_mask = doc_masks[i:i+1]

            # 计算相似度矩阵
            similarity_matrix = torch.matmul(query_emb, doc_emb.unsqueeze(0).transpose(-2, -1))

            # 应用文档mask
            doc_mask_expanded = doc_mask.unsqueeze(1)
            similarity_matrix = similarity_matrix.masked_fill(~doc_mask_expanded.bool(), -1e9)

            # MaxSim操作
            max_sim_per_query_token = similarity_matrix.max(dim=-1)[0]

            # 应用查询mask
            query_mask_expanded = query_mask.unsqueeze(0)
            max_sim_per_query_token = max_sim_per_query_token.masked_fill(~query_mask_expanded.bool(), 0)

            # 求和得到最终分数
            colbert_score = max_sim_per_query_token.sum(dim=-1).item()
            scores.append(colbert_score)

        return scores

    def compress_documents(
        self,
        documents: Sequence[Document],
        query: str,
        callbacks=None,
    ) -> Sequence[Document]:
        """对文档进行ColBERT重排序"""
        if len(documents) == 0:
            return documents

        # 编码查询
        query_inputs = self.tokenizer(
            [query],
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128
        )

        with torch.no_grad():
            query_outputs = self.model(**query_inputs)
            query_embeddings = F.normalize(query_outputs.last_hidden_state, p=2, dim=-1)

        # 编码文档
        doc_texts = [doc.page_content for doc in documents]
        doc_inputs = self.tokenizer(
            doc_texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128
        )

        with torch.no_grad():
            doc_outputs = self.model(**doc_inputs)
            doc_embeddings = F.normalize(doc_outputs.last_hidden_state, p=2, dim=-1)

        # 计算ColBERT相似度
        scores = self.calculate_colbert_similarity(
            query_embeddings,
            doc_embeddings,
            query_inputs['attention_mask'],
            doc_inputs['attention_mask']
        )

        # 排序并返回前5个
        scored_docs = list(zip(documents, scores))
        scored_docs.sort(key=lambda x: x[1], reverse=True)
        reranked_docs = [doc for doc, _ in scored_docs[:5]]

        return reranked_docs
        


## 初始化配置

In [4]:
embedding_model = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-large-zh-v1.5")

llm = ChatDeepSeek(
    model="deepseek-chat", 
    temperature=0.1, 
    api_key=os.getenv("DEEPSEEK_API_KEY")
)

/tmp/ipykernel_60168/604755049.py:1: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-large-zh-v1.5")


## 加载和处理文档

In [5]:
from pathlib import Path
# 如果在 Notebook 中运行，__file__ 不存在，改用当前工作目录
try:
    SCRIPT_DIR = Path(__file__).resolve().parent
except NameError:
    SCRIPT_DIR = Path.cwd()  # 使用当前工作目录

loader = TextLoader(SCRIPT_DIR / "../../data/C4/txt/ai.txt", encoding = "utf-8")
docs = loader.load()
text_spliter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunk_docs = text_spliter.split_documents(docs)

## 创建向量存储和基础检索器

In [6]:
vector_stores = FAISS.from_documents(chunk_docs, embedding_model)
base_retriever = vector_stores.as_retriever(search_kwargs={"k": 20})

## 设置ColBERT重排序器

In [7]:
reranker = ColBERTReranker()

ColBERT模型加载完成


## 设置LLM压缩器

In [8]:
compressor = LLMChainExtractor.from_llm(llm)

## 组装压缩管道

In [9]:
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[reranker, compressor]
)

## 创建最终的压缩检索器

In [10]:
final_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor,
    base_retriever=base_retriever
)

## 执行查询并展示结果

In [12]:
query = "AI还有哪些缺陷需要克服？"
print(f"\n{'='*20} 开始执行查询 {'='*20}")
print(f"查询: {query}\n")

# 7.1 基础检索结果
print(f"--- (1) 基础检索结果 (Top 20) ---")
base_results = base_retriever.get_relevant_documents(query)
for i, doc in enumerate(base_results):
    print(f"  [{i+1}] {doc.page_content[:100]}...\n")


==================== 开始执行查询 ====================
查询: AI还有哪些缺陷需要克服？

--- (1) 基础检索结果 (Top 20) ---


/tmp/ipykernel_60168/232063080.py:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  base_results = base_retriever.get_relevant_documents(query)


  [1] 行业巨头谷歌公司也没闲着。该公司在5月推出整体性能和智能推理能力均较以往版本大幅提升的多个“双子座2.5”系列模型，并发布了多个多模态模型，如图像生成模型Imagen 4和视频生成模型Veo 3，具备...

  [2] 一个比较明显的问题是，AI生成内容虽然已非常流畅，但提供的信息很多时候还是不准确。5月，日本研究人员在德国《先进科学》杂志发表的一项研究成果中指出，这一问题与人类的语言障碍——失语症类似。

    ...

  [3] 业界也确实在努力从不同角度去寻求优化大模型的解决方案。中国科学院自动化研究所联合鹏城实验室提出了一种高效推理策略AutoThink，可让大模型实现自主切换思考模式，避免“过度思考”。

    据研究...

  [4] 一些国家已在积极尝试通过优化政策、法规来营造更好的AI创新环境。日本参议院全体会议5月28日以多数赞成票通过该国首部专门针对AI的法律，旨在促进AI相关技术研发和应用并防止其滥用。依据这部《人工智能相...

  [5] 5月，全球多家科技公司发布新的大模型，它们在语义理解、多模态等方面进一步提升，人工智能（AI）的能力边界在不断扩大。随着无人驾驶、机器人等技术借助AI快速进化并逐步投入市场，不少国家通过推进法规建设、...



In [13]:
# 7.2 使用管道压缩器的最终结果
print(f"\n--- (2) 管道压缩后结果 (ColBERT重排 + LLM压缩) ---")
final_results = final_retriever.get_relevant_documents(query)
for i, doc in enumerate(final_results):
    print(f"  [{i+1}] {doc.page_content}\n")


--- (2) 管道压缩后结果 (ColBERT重排 + LLM压缩) ---
  [1] 一个比较明显的问题是，AI生成内容虽然已非常流畅，但提供的信息很多时候还是不准确。

  [2] 中国科学院自动化研究所联合鹏城实验室提出了一种高效推理策略AutoThink，可让大模型实现自主切换思考模式，避免“过度思考”。

  [3] AI仍有不少缺陷需克服

    尽管当前AI应用已相当广泛，但不少缺陷还是会影响其实用性。研究人员正努力分析导致这些缺陷的原因并寻求新的解决方法，从而改善AI的性能。

    一个比较明显的问题是，AI生成内容虽然已非常流畅，但提供的信息很多时候还是不准确。5月，日本研究人员在德国《先进科学》杂志发表的一项研究成果中指出，这一问题与人类的语言障碍——失语症类似。

